In [1]:
import LeakDetector
import json
import glob
from os.path import basename
import pandas as pd
from urllib.parse import urlparse
from pandarallel import pandarallel
from tld import get_fld
import tldextract
import os

## Function that needed

In [2]:
def search_list(*search):
    """
    check whether the value of *search is exist and make a list of it

    :return list: list of search that is not None or ''
    """
    return [i for i in search if i is not None and i != '']

In [3]:
def selection_for_search(final_url):
    """
    parse url using urlparse, check whether the value exist and make a list of it

    :param url final_url: url to be parsed
    :return list: list of 
    """
    netloc_search = urlparse(final_url).netloc
    hostname_search = urlparse(final_url).hostname
    path_search = urlparse(final_url).path
    if path_search == '/':
        path_search = ""
    params_search = urlparse(final_url).params
    query_search = urlparse(final_url).query
    
    return search_list(netloc_search, hostname_search, path_search, params_search, query_search)


In [4]:
def leaky(row):
 
    final_url = row['final_url']
    req_url = row['req_url']
    req_data = row["post_data"]
    ref_data = row["ref_data"]
        
    search_terms = selection_for_search(final_url)

    leak_detector = LeakDetector.LeakDetector(
                search_terms, 
                encoding_set=LeakDetector.ENCODINGS_NO_ROT,
                hash_set=LeakDetector.LIKELY_HASHES,
                encoding_layers=3,
                hash_layers=3,
                debugging=False
                )

    url_leaks = None
    post_leaks = None
    referer_leaks = None
    try:
        valid_url = get_fld(final_url)
    except Exception as e:
        print("ERROR: ", final_url, e)
        return None, None, None        
    for __ in range(1):
        if (req_url.startswith('blob:') or req_url.startswith('chromewebdata') or req_url.startswith('chrome-extension:') or req_url.startswith('localhost')):
            continue
        try:
            if not tldextract.extract(req_url).suffix:
                continue
        except Exception as e:
            print("ERROR: ", req_url, e)
            continue
        if get_fld(req_url) != valid_url:
            try:
                url_leaks = leak_detector.check_url(req_url, encoding_layers=3) 
            except Exception as err:
                # print("ERROR: Cannot do url leak detector", req_url, json_name, err)
                pass
            try:
                post_leaks = leak_detector.check_post_data(req_data, encoding_layers=3)
            except Exception as err:
                # print("ERROR: Cannot do post leak detector", req_data, json_name, err)
                pass
            try:
                referer_leaks = leak_detector.check_url(ref_data, encoding_layers=3)
            except Exception as err:
                pass

    return str(url_leaks), str(post_leaks), str(referer_leaks)

## Open Folder and extract JSON file

In [5]:
# folder_path = "2023-10-17_inner_collector/*.json"  # replace with the path to your folder
# folder_path = "2023-10-11_inner_collector/*.json"  # replace with the path to your folder
# folder_path = "2023-10-11_inner_collector/osheaga.com_5288.json"  # replace with the path to your folder
# folder_path = "2023-10-11_inner_collector/an.gov.br_ea4d.json"
# folder_path = '2023-10-11_inner_collector/versatel.de_b202.json'
# folder_path = '2023-10-11_inner_collector/postman-echo.com_0eb5.json'
# folder_path = '2023-11-29_inner_collector3/*.json'
folder_path = '2023-12-08_inner_collector_optIn/*.json'

extract_date = folder_path[:10]
extract_type = folder_path[folder_path.rfind("_")+1:folder_path.rfind("/")]

print("folder to extract: " + folder_path)
print("type of cmps: " + extract_type)

# ParallelPandas.initialize(n_cpu=8, split_factor=4, disable_pr_bar=False)
pandarallel.initialize(nb_workers=8, progress_bar=True)

extracted_list = []
# count = 0

for json_path in glob.glob(folder_path):
    json_name = basename(json_path)

    try:
        with open(json_path, encoding='utf-8-sig') as file:
            results = json.load(file)
    except Exception as e:
        print("ERROR: Cannot load the json", json_name, e)
        continue
    
    try:
        init_url = results["initialUrl"]
        final_url = results["finalUrl"]
    except Exception as e:
        print("ERROR: Cannot find the url", json_name, e)
        continue
    
    results_data = results["data"]

    for req in results_data ["requests"]:
        req_url = req["url"]
        req_method = req.get("method")
        post_data = req.get("postData")
        ref_pol_data = req.get('reqReferrerPolicy')
        ref_data = req.get('requestHeaders', {}).get('referer', '')
        extracted_details = [init_url, final_url, req_url, req_method,  ref_pol_data , ref_data, post_data]
        extracted_list.append(extracted_details)

extractedDF = pd.DataFrame(extracted_list, columns=["init_url", "final_url", "req_url", "req_method","ref_pol_data" ,"ref_data", "post_data"])
extractedDF


folder to extract: 2023-12-08_inner_collector_optIn/*.json
type of cmps: optIn
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
ERROR: Cannot find the url metadata.json 'initialUrl'


,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,post_data
0,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,GET,strict-origin-when-cross-origin,,
1,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,
2,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,
3,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,
4,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://fonts.googleapis.com/css?family=PT+San...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,
...,...,...,...,...,...,...,...
518323,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/u/?gdpr=0&oid=207...,GET,strict-origin-when-cross-origin,,
518324,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=cro&oid=2...,GET,strict-origin-when-cross-origin,,
518325,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://id5-sync.com/g/v2/1270.json,POST,strict-origin-when-cross-origin,https://6015542.fls.doubleclick.net/,"{""partner"":1270,""v"":""1.0.36"",""o"":""api"",""tml"":""..."
518326,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=ttd&oid=2...,GET,strict-origin-when-cross-origin,,


## Run Leak Detector

In [6]:
# run leak detector and put it into the colomn in dataframe
extractedDF[['url_leaks','post_leaks','referer_leaks']] = extractedDF.parallel_apply(leaky, axis=1, result_type='expand')


Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse u

In [7]:
extractedDF

,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,post_data,url_leaks,post_leaks,referer_leaks
0,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,GET,strict-origin-when-cross-origin,,,None,None,None
1,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None
2,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None
3,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None
4,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://fonts.googleapis.com/css?family=PT+San...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,[],[],[]
...,...,...,...,...,...,...,...,...,...,...
518323,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/u/?gdpr=0&oid=207...,GET,strict-origin-when-cross-origin,,,[],[],[]
518324,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=cro&oid=2...,GET,strict-origin-when-cross-origin,,,[],[],[]
518325,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://id5-sync.com/g/v2/1270.json,POST,strict-origin-when-cross-origin,https://6015542.fls.doubleclick.net/,"{""partner"":1270,""v"":""1.0.36"",""o"":""api"",""tml"":""...",[],"[('www.andapp.jp',)]",[]
518326,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=ttd&oid=2...,GET,strict-origin-when-cross-origin,,,[],[],[]


In [8]:
# filter the leaks and put it into a new colomn in dataframe 

extractedDF['leaks'] = extractedDF.parallel_apply(lambda row: ' - '.join([col for col in ['url_leaks', 'post_leaks', 'referer_leaks'] if row[col] not in [None, '[]', 'None' ]]), axis=1)

In [9]:
extractedDF

,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,post_data,url_leaks,post_leaks,referer_leaks,leaks
0,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,GET,strict-origin-when-cross-origin,,,None,None,None,
1,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None,
2,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None,
3,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.ox.ac.uk/sites/files/oxford/advagg...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/research,,None,None,None,
4,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://fonts.googleapis.com/css?family=PT+San...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,[],[],[],
...,...,...,...,...,...,...,...,...,...,...,...
518323,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/u/?gdpr=0&oid=207...,GET,strict-origin-when-cross-origin,,,[],[],[],
518324,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=cro&oid=2...,GET,strict-origin-when-cross-origin,,,[],[],[],
518325,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://id5-sync.com/g/v2/1270.json,POST,strict-origin-when-cross-origin,https://6015542.fls.doubleclick.net/,"{""partner"":1270,""v"":""1.0.36"",""o"":""api"",""tml"":""...",[],"[('www.andapp.jp',)]",[],post_leaks
518326,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/c/map/?sp=ttd&oid=2...,GET,strict-origin-when-cross-origin,,,[],[],[],


In [10]:
# Create the directory if it doesn't exist
os.makedirs("data_raw", exist_ok=True)

# Save the file into the 'data_raw' directory
extractedDF.to_csv("data_raw/"+extract_date+"_"+extract_type+'_raw.csv', index=False)

In [11]:
# only show the data if leaks is not empty
extractedDF_print = extractedDF[extractedDF['leaks'] != '']
extractedDF_print

,init_url,final_url,req_url,req_method,ref_pol_data,ref_data,post_data,url_leaks,post_leaks,referer_leaks,leaks
108,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.youtube.com/embed/VQwYwDqHA7I?wmod...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,"[('urlencode', 'www.ox.ac.uk'), ('www.ox.ac.uk...",[],[],url_leaks
110,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://px.ads.linkedin.com/collect?v=2&fmt=js...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,"[('urlencode', 'www.ox.ac.uk'), ('urlencode', ...",[],[],url_leaks
111,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://px.ads.linkedin.com/collect?v=2&fmt=js...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,"[('urlencode', 'www.ox.ac.uk'), ('urlencode', ...",[],[],url_leaks
115,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://apikeys.civiccomputing.com/c/v?d=www.o...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,"[('urlencode', 'www.ox.ac.uk'), ('www.ox.ac.uk...",[],[],url_leaks
116,https://www.ox.ac.uk/research,https://www.ox.ac.uk/research,https://www.linkedin.com/px/li_sync?redirect=h...,GET,strict-origin-when-cross-origin,https://www.ox.ac.uk/,,"[('urlencode', 'www.ox.ac.uk'), ('www.ox.ac.uk...",[],[],url_leaks
...,...,...,...,...,...,...,...,...,...,...,...
518298,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://syndication.twitter.com/i/jot/embeds?l...,GET,strict-origin-when-cross-origin,https://www.andapp.jp/,,"[('urlencode', 'www.andapp.jp'), ('urlencode',...",[],[],url_leaks
518302,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://www.google.com/pagead/1p-user-list/878...,GET,strict-origin-when-cross-origin,https://www.andapp.jp/,,"[('urlencode', 'www.andapp.jp'), ('urlencode',...",[],[],url_leaks
518304,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://www.youtube.com/youtubei/v1/log_event?...,POST,strict-origin-when-cross-origin,https://www.youtube.com/embed/19s0G5lLc0g?rel=...,"{""context"":{""client"":{""hl"":""en"",""gl"":""US"",""cli...",[],"[('www.andapp.jp',)]","[('urlencode', 'www.andapp.jp'), ('www.andapp....",post_leaks - referer_leaks
518317,https://www.mbga.jp/_game_intro?game_id=12016007,https://www.andapp.jp/apps/12016007?from=www_m...,https://aw.dw.impact-ad.jp/ut/rep?u=2714&v=2&r...,GET,strict-origin-when-cross-origin,https://6015542.fls.doubleclick.net/,,"[('urlencode', 'www.andapp.jp'), ('www.andapp....",[],[],url_leaks


In [12]:
# print all of the values if the colomn leaks in not null to a csv file
extractedDF_print.to_csv("data_raw/"+extract_date+"_"+extract_type+'_leaks_raw.csv', index=False)